# **Exploratory Data Analysis** of Independent Features in **PlantTraits2024**: MODIS vs. VOD

## 1. Wczytanie danych i wstępna eksploracja

### Kroki:
* Załadowanie zbiorów treningowego i testowego.
* Sprawdzenie liczby kolumn i wierszy w obu zbiorach.
* Sprawdzenie duplikatów → czy mamy powielone wiersze?
* Podstawowe statystyki (średnia, min, max, percentyle) dla pierwszego rozeznania w wartościach.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
pd.set_option('display.max_columns', 200)

In [12]:
train_csv = "I:/Jacob/Documents/PlantTraits2024/data/train.csv"
test_csv = "I:/Jacob/Documents/PlantTraits2024/data/test.csv"

train_data = pd.read_csv(train_csv)
test_data = pd.read_csv(test_csv)

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55489 entries, 0 to 55488
Columns: 176 entries, id to X3112_sd
dtypes: float64(54), int64(122)
memory usage: 74.5 MB


***
Mamy 55489 wierszy (próbek) oraz 176 kolumn
***

In [5]:
all_without_id = [col for col in train_data.columns if col != 'id']
num_duplicates = train_data[all_without_id].duplicated().sum()
print("Liczba zduplikowanych wierszy:", num_duplicates)

Liczba zduplikowanych wierszy: 4200


***
W celu sprawdzenia duplikatów odrzucamy kolumnę id. Liczba duplikatów zawarta w całym zbiorze wynosi 4200. Poniżej zaprezentowano również podstawowe statystki zbioru dla obu zbiorów - treningowego i testowego.
***

In [7]:
train_data.describe()

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,SOIL_bdod_30.60cm_mean_0.01_deg,SOIL_bdod_5.15cm_mean_0.01_deg,SOIL_bdod_60.100cm_mean_0.01_deg,SOIL_cec_0.5cm_mean_0.01_deg,SOIL_cec_100.200cm_mean_0.01_deg,SOIL_cec_15.30cm_mean_0.01_deg,SOIL_cec_30.60cm_mean_0.01_deg,SOIL_cec_5.15cm_mean_0.01_deg,SOIL_cec_60.100cm_mean_0.01_deg,SOIL_cfvo_0.5cm_mean_0.01_deg,SOIL_cfvo_100.200cm_mean_0.01_deg,SOIL_cfvo_15.30cm_mean_0.01_deg,SOIL_cfvo_30.60cm_mean_0.01_deg,SOIL_cfvo_5.15cm_mean_0.01_deg,SOIL_cfvo_60.100cm_mean_0.01_deg,SOIL_clay_0.5cm_mean_0.01_deg,SOIL_clay_100.200cm_mean_0.01_deg,SOIL_clay_15.30cm_mean_0.01_deg,SOIL_clay_30.60cm_mean_0.01_deg,SOIL_clay_5.15cm_mean_0.01_deg,SOIL_clay_60.100cm_mean_0.01_deg,SOIL_nitrogen_0.5cm_mean_0.01_deg,SOIL_nitrogen_100.200cm_mean_0.01_deg,SOIL_nitrogen_15.30cm_mean_0.01_deg,SOIL_nitrogen_30.60cm_mean_0.01_deg,SOIL_nitrogen_5.15cm_mean_0.01_deg,SOIL_nitrogen_60.100cm_mean_0.01_deg,SOIL_ocd_0.5cm_mean_0.01_deg,SOIL_ocd_100.200cm_mean_0.01_deg,SOIL_ocd_15.30cm_mean_0.01_deg,SOIL_ocd_30.60cm_mean_0.01_deg,SOIL_ocd_5.15cm_mean_0.01_deg,SOIL_ocd_60.100cm_mean_0.01_deg,SOIL_ocs_0.30cm_mean_0.01_deg,SOIL_phh2o_0.5cm_mean_0.01_deg,SOIL_phh2o_100.200cm_mean_0.01_deg,SOIL_phh2o_15.30cm_mean_0.01_deg,SOIL_phh2o_30.60cm_mean_0.01_deg,SOIL_phh2o_5.15cm_mean_0.01_deg,SOIL_phh2o_60.100cm_mean_0.01_deg,SOIL_sand_0.5cm_mean_0.01_deg,SOIL_sand_100.200cm_mean_0.01_deg,SOIL_sand_15.30cm_mean_0.01_deg,SOIL_sand_30.60cm_mean_0.01_deg,SOIL_sand_5.15cm_mean_0.01_deg,SOIL_sand_60.100cm_mean_0.01_deg,SOIL_silt_0.5cm_mean_0.01_deg,SOIL_silt_100.200cm_mean_0.01_deg,SOIL_silt_15.30cm_mean_0.01_deg,SOIL_silt_30.60cm_mean_0.01_deg,SOIL_silt_5.15cm_mean_0.01_deg,SOIL_silt_60.100cm_mean_0.01_deg,SOIL_soc_0.5cm_mean_0.01_deg,SOIL_soc_100.200cm_mean_0.01_deg,SOIL_soc_15.30cm_mean_0.01_deg,SOIL_soc_30.60cm_mean_0.01_deg,SOIL_soc_5.15cm_mean_0.01_deg,SOIL_soc_60.100cm_mean_0.01_deg,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_ban

In [9]:
test_data.describe()

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,SOIL_bdod_30.60cm_mean_0.01_deg,SOIL_bdod_5.15cm_mean_0.01_deg,SOIL_bdod_60.100cm_mean_0.01_deg,SOIL_cec_0.5cm_mean_0.01_deg,SOIL_cec_100.200cm_mean_0.01_deg,SOIL_cec_15.30cm_mean_0.01_deg,SOIL_cec_30.60cm_mean_0.01_deg,SOIL_cec_5.15cm_mean_0.01_deg,SOIL_cec_60.100cm_mean_0.01_deg,SOIL_cfvo_0.5cm_mean_0.01_deg,SOIL_cfvo_100.200cm_mean_0.01_deg,SOIL_cfvo_15.30cm_mean_0.01_deg,SOIL_cfvo_30.60cm_mean_0.01_deg,SOIL_cfvo_5.15cm_mean_0.01_deg,SOIL_cfvo_60.100cm_mean_0.01_deg,SOIL_clay_0.5cm_mean_0.01_deg,SOIL_clay_100.200cm_mean_0.01_deg,SOIL_clay_15.30cm_mean_0.01_deg,SOIL_clay_30.60cm_mean_0.01_deg,SOIL_clay_5.15cm_mean_0.01_deg,SOIL_clay_60.100cm_mean_0.01_deg,SOIL_nitrogen_0.5cm_mean_0.01_deg,SOIL_nitrogen_100.200cm_mean_0.01_deg,SOIL_nitrogen_15.30cm_mean_0.01_deg,SOIL_nitrogen_30.60cm_mean_0.01_deg,SOIL_nitrogen_5.15cm_mean_0.01_deg,SOIL_nitrogen_60.100cm_mean_0.01_deg,SOIL_ocd_0.5cm_mean_0.01_deg,SOIL_ocd_100.200cm_mean_0.01_deg,SOIL_ocd_15.30cm_mean_0.01_deg,SOIL_ocd_30.60cm_mean_0.01_deg,SOIL_ocd_5.15cm_mean_0.01_deg,SOIL_ocd_60.100cm_mean_0.01_deg,SOIL_ocs_0.30cm_mean_0.01_deg,SOIL_phh2o_0.5cm_mean_0.01_deg,SOIL_phh2o_100.200cm_mean_0.01_deg,SOIL_phh2o_15.30cm_mean_0.01_deg,SOIL_phh2o_30.60cm_mean_0.01_deg,SOIL_phh2o_5.15cm_mean_0.01_deg,SOIL_phh2o_60.100cm_mean_0.01_deg,SOIL_sand_0.5cm_mean_0.01_deg,SOIL_sand_100.200cm_mean_0.01_deg,SOIL_sand_15.30cm_mean_0.01_deg,SOIL_sand_30.60cm_mean_0.01_deg,SOIL_sand_5.15cm_mean_0.01_deg,SOIL_sand_60.100cm_mean_0.01_deg,SOIL_silt_0.5cm_mean_0.01_deg,SOIL_silt_100.200cm_mean_0.01_deg,SOIL_silt_15.30cm_mean_0.01_deg,SOIL_silt_30.60cm_mean_0.01_deg,SOIL_silt_5.15cm_mean_0.01_deg,SOIL_silt_60.100cm_mean_0.01_deg,SOIL_soc_0.5cm_mean_0.01_deg,SOIL_soc_100.200cm_mean_0.01_deg,SOIL_soc_15.30cm_mean_0.01_deg,SOIL_soc_30.60cm_mean_0.01_deg,SOIL_soc_5.15cm_mean_0.01_deg,SOIL_soc_60.100cm_mean_0.01_deg,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_ban

### Porównanie liczb kolumn train i test
Zauważamy, że dane treningowe mają więcej kolumn niż testowe

In [15]:
train_columns = set(train_data.columns)
test_columns = set(test_csv.columns)

only_in_train = train_columns - test_columns

print("Kolumny tylko w train: ", only_in_train)

AttributeError: 'str' object has no attribute 'columns'

Te kolumny to są nasze zmienne zależne, które musimy przewidzieć.

## 2. Wyodrębnienie danych MODIS i VOD

### Kroki:

* Filtrowanie kolumn związanych z MODIS i VOD.
* Podstawowe statystyki dla obu grup cech (min, max, rozkład wartości).
* Sprawdzenie liczby miesięcy w każdej grupie danych (czy mamy kompletne miesiące?).


In [16]:
modis_vod_cols = [col for col in train_data.columns if col.startswith("MODIS_") or col.startswith("VOD_")]

df_modis_vod = train_data[modis_vod_cols].copy()

***
Szybki podgląd
***

In [17]:
df_modis_vod.head()

,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m1,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m10,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m11,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m12,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m2,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m3,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m3,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m3,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m3,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m3,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m4,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m4,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m4,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m4,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m4,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m5,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m5,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m5,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m5,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m5,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m6,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m6,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m6,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m6,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m6,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m7,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m7,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m7,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m7,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m7,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m8,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m8,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m8,MODIS_2000.2020_monthly_mean_surface_reflectance_band_04_._month_m8,MODIS_2000.2020_monthly_mean_surface_reflectance_band_05_._month_m8,MODIS_2000.2020_monthly_mean_surface_reflectance_band_01_._month_m9,MODIS_2000.2020_monthly_mean_surface_reflectance_band_02_._month_m9,MODIS_2000.2020_monthly_mean_surface_reflectance_band_03_._month_m9,MODIS_2000.2020_monthly_mean_surface_ref

In [ ]:
df_modis_vod.shape

In [ ]:
df_modis_vod.describe()

### Sprawdzenie wartości NaN i null w podzbiorze modis_vod

In [ ]:
df_modis_vod.isnull().sum().sum()

### Duplikaty

In [ ]:
num_duplicates = df_modis_vod.duplicated().sum()
print("Liczba zduplikowanych wierszy:", num_duplicates)

W podzbiorze MODIS zidentyfikowano 18512 duplikatów, jednak ich usunięcie nie jest możliwe bez analizy pełnego zbioru danych. Ich obecność może być wynikiem kontekstu lub relacji z innymi danymi.

## 3. Analiza wartości VOD
### Kroki:
* Wartości max i min występujące w podzbiorze kolumn VOD.
* Histogramy.
* Analiza korelacji.
* Wykres liniowy na przestrzenie miesięcy dla każdego z pasm.

In [ ]:
df_vod   = df_modis_vod[[c for c in df_modis_vod.columns if c.startswith('VOD_')]]

print("Wartość minimalna:", df_vod.min().min())
print("Wartość maksymalna:", df_vod.max().max())

Sprawdzenie czy występują wartości ujemne

In [ ]:
negative_mask = (df_vod < 0)
any_negatives = negative_mask.any().any()
print("Czy istnieją wartości ujemne w VOD?", any_negatives)

Podstawowe statystki oraz podgląd na kolumny df_modis_vod pozwoliły na wyróżnienie trzech pasm - C, X i Ku oraz 12 miesięcy, co łączeni daje nam 36 kolumn.


In [ ]:
df_vod.hist(figsize=(15, 10), bins=30)
plt.tight_layout()
plt.show()

Dla lepszego zobrazowania zastosowano wykres "boxplot" z biblioteki seaborn.

In [ ]:
plt.figure(figsize=(15, 8))
sns.boxplot(data=df_vod, orient='h')
plt.show()

W trakcie eksploracyjnej analizy danych (EDA) zdecydowano o pozostawieniu wszystkich obserwacji w kolumnach VOD, mimo identyfikacji wartości skrajnych, takich jak minimalna (approx. 0.00069) i maksymalna (approx. 1.8689). Zamiast usuwania wartości skrajnych, podjęto decyzję o ich standaryzacji. Poniżej przedstawiono uzasadnienie tej decyzji:

1. Wartości ujemne w danych VOD byłyby błędem technicznym lub artefaktem obliczeń, co mogłoby uzasadniać ich usunięcie. Jednakże w analizowanym zbiorze danych nie występują wartości ujemne, co potwierdza poprawność danych w tym zakresie.
2. Wartości bliskie 0 w VOD mogą być reprezentatywne dla obszarów pustynnych lub innych środowisk o niskiej zawartości biomasy.
3. Wartości maksymalne, takie jak 1.86, mogą odzwierciedlać obszary o bardzo wysokiej zawartości wody w roślinności, np. wilgotne lasy tropikalne. Ich obecność może być rzeczywista i wartościowa.
4. Usuwanie wartości skrajnych powinno być uzasadnione wiedzą domenową lub techniczną wskazującą, że są one błędne. W tym przypadku nie ma dowodów sugerujących, że wartości skrajne w VOD są efektem błędu pomiarowego czy obliczeniowego.
5. Zamiast usuwania wartości skrajnych, w poźniejszym etapie prawdopodobnie zastosuje standaryzację, co pozwali zachować informacje zawarte w danych, jednocześnie ograniczając ich wpływ na modele predykcyjne i dalszą analizę.


### Analiza Korelacji

### Na początek korelacja między różnymi pasmami i miesiącami

In [ ]:
corr = df_vod.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, cmap='viridis')
plt.title("Macierz korelacji VOD (pasma i miesiące)")
plt.show()

Na tym etapie widać już jakąś korelację dla pasm C i X.

### Porównanie ogólnego rozkładu pasm

In [ ]:
# Porównanie ogólnego rozkładu pasm
vod_c_cols = [c for c in df_vod.columns if c.startswith('VOD_C')]
df_vod_c = df_vod[vod_c_cols]  # DataFrame samych kolumn C
df_vod_c_melt = df_vod_c.melt(var_name="col_name", value_name="vod_value")
df_vod_c_melt["band"] = "C"

vod_ku_cols = [c for c in df_vod.columns if c.startswith('VOD_Ku')]
df_vod_ku = df_vod[vod_ku_cols]
df_vod_ku_melt = df_vod_ku.melt(var_name="col_name", value_name="vod_value")
df_vod_ku_melt["band"] = "Ku"

vod_x_cols = [c for c in df_vod.columns if c.startswith('VOD_X')]
df_vod_x = df_vod[vod_x_cols]
df_vod_x_melt = df_vod_x.melt(var_name="col_name", value_name="vod_value")
df_vod_x_melt["band"] = "X"

df_vod_bands = pd.concat([df_vod_c_melt, df_vod_ku_melt, df_vod_x_melt], ignore_index=True)

plt.figure(figsize=(8, 6))
sns.boxplot(data=df_vod_bands, x="band", y="vod_value")
plt.title("Porównanie rozkładów VOD w pasmach C, Ku i X (wszystkie miesiące razem)")
plt.show()

Dla pasma C i X widzimy bardzo zbliżony rozkład wartości VOD, co może sugerować podobne charakterystyki rejestrowanych danych w tych zakresach częstotliwości. Natomiast pasmo Ku (12–18 GHz) znacząco różni się swoim rozkładem, co można zauważyć przez większą rozpiętość wartości oraz większą liczbę wartości wychodzących poza tzw. zakres międzykwartylowy (IQR). W szczególności w paśmie Ku obserwujemy znacznie więcej wartości odstających, które mogą reprezentować ekstremalne warunki środowiskowe (np. tropikalne lasy o bardzo wysokiej wilgotności lub pustynne obszary o minimalnym poziomie wilgotności).

### Wykres liniowy dla średniej wartości VOD w kolejnych miesiącach dla każdego pasma

In [ ]:
bands = ["VOD_C", "VOD_Ku", "VOD_X"]
months = range(1, 13)

plt.figure(figsize=(10, 6))

for band in bands:
    avg_month = []

    for m in months:
        col_suffix = f"_m{m:02d}"
        month_cols = [c for c in df_vod.columns if band in c and col_suffix in c]

        if len(month_cols) == 0:
            avg_month.append(None)
            continue

        col_data = df_vod[month_cols[0]]
        mean_val = col_data.mean(skipna=True)
        avg_month.append(mean_val)

    plt.plot(months, avg_month, marker='o', label=band)

plt.xticks(months)  # miesiące na osi X
plt.xlabel("Miesiąc")
plt.ylabel("Średnia wartość VOD")
plt.title("Sezonowe wahania VOD w poszczególnych pasmach")
plt.legend()
plt.show()

***
Widać, że obserwowany jest wzrost średniej VOD dla każdego pasma w cieplejszych miesiącach oraz podobieństwo średnich cech dla C i X. Można się zastanowić nad scaleniem pasma C i X i sprawdzić jaki wpływ ma to na model.
***

### Heatmapa oraz macierz korelacji dla pasma C i X w celu lepszego zobrazowania ich powiązań

In [ ]:
vod_c_cols = [col for col in df_modis_vod.columns if col.startswith('VOD_C')]
vod_x_cols = [col for col in df_modis_vod.columns if col.startswith('VOD_X')]

corr_matrix_c_to_x = df_modis_vod[vod_c_cols + vod_x_cols].corr()

print(corr_matrix_c_to_x)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix_c_to_x, cmap='viridis')
plt.title("Macierz korelacji VOD_X i VOD_C")
plt.show()

## 4. Scalanie pasm C i X w VOD, aby zobaczyć jak może to wyglądac w preprocessingu

Dlaczego warto to zrobić?
Silna korelacja (approx. 0.96) między C i X oznacza, że wartości mają niemal te same wartości.
Scalenie i uśrednienie ich wartości pozwoli na redukcję wymiarów bez utrarty informacji.
Dla każdego miesiąca tworzymy nową skumulowaną kolumne "CX".

In [1]:
df_modis_vod_cx = df_modis_vod

months = [f"m{str(i).zfill(2)}" for i in range(1, 13)]

for month in months:
    vod_c_col = next((col for col in df_modis_vod_cx.columns if f"VOD_C" in col and month in col), None)
    vod_x_col = next((col for col in df_modis_vod_cx.columns if f"VOD_X" in col and month in col), None)

    if vod_c_col and vod_x_col:
        df_modis_vod_cx[f"VOD_CX_{month}"] = df_modis_vod_cx[[vod_c_col, vod_x_col]].mean(axis=1)

        df_modis_vod_cx.drop(columns=[vod_c_col, vod_x_col], inplace=True)

df_modis_vod_cx.head()

NameError: name 'df_modis_vod' is not defined

***
Powyżej podgląd po scaleniu i drop na X i C.
***

### Zawężenie zakresu (osi czasu) do pór roku.

In [ ]:
df_modis_vod_seasons = df_modis_vod_cx

seasons = {
    "Winter": ["m12", "m01", "m02"],
    "Spring": ["m03", "m04", "m05"],
    "Summer": ["m06", "m07", "m08"],
    "Autumn": ["m09", "m10", "m11"]
}

for season, months in seasons.items():
    df_modis_vod_seasons[f"VOD_CX_{season}"] = df_modis_vod_seasons[[f"VOD_CX_{m}" for m in months]].mean(axis=1)
    df_modis_vod_seasons[f"VOD_Ku_{season}"] = df_modis_vod_seasons[[f"VOD_Ku_1987_2017_multiyear_mean_{m}" for m in months]].mean(axis=1)

df_modis_vod_seasons.drop(columns=[col for col in df_modis_vod_seasons.columns if any(m in col for m in sum(seasons.values(), []))], inplace=True)

df_modis_vod_seasons.head()

### Wizualizacja

In [ ]:
seasonal_means = df_modis_vod_seasons[["VOD_CX_Winter", "VOD_CX_Spring", "VOD_CX_Summer", "VOD_CX_Autumn",
                      "VOD_Ku_Winter", "VOD_Ku_Spring", "VOD_Ku_Summer", "VOD_Ku_Autumn"]].mean()

# Wykres
plt.figure(figsize=(10, 5))
sns.barplot(x=seasonal_means.index, y=seasonal_means.values, palette="viridis")
plt.title("Średnie wartości VOD dla pór roku")
plt.ylabel("Średnia wartość VOD")
plt.xticks(rotation=45)
plt.show()

### Wykres liniowy tym razem dla pór roku i dwóch pasm VOD

In [ ]:
bands = ["VOD_CX", "VOD_Ku"]
seasons = ["Winter", "Spring", "Summer", "Autumn"]

plt.figure(figsize=(10, 6))

for band in bands:
    avg_season = []

    for s in seasons:
        season_cols = [c for c in df_modis_vod_seasons.columns if band in c and s in c]

        if len(season_cols) == 0:
            avg_season.append(None)
            continue

        col_data = df_modis_vod_seasons[season_cols[0]]
        mean_val = col_data.mean(skipna=True)
        avg_season.append(mean_val)

    plt.plot(seasons, avg_season, marker='o', label=band)

print(avg_season)
plt.xticks(seasons)  # miesiące na osi X
plt.xlabel("Pora roku")
plt.ylabel("Średnia wartość VOD")
plt.title("Sezonowe wahania VOD w poszczególnych pasmach")
plt.legend()
plt.show()

## 5. Analiza wartości MODIS
### Kroki:
* Sprawdzenie wartości ujemnnych oraz zapoznanie się ze strukturą.
* Struktura pasm.
* Wykres liniowy.
* Macierz korelacji.

Sprawdzenie czy występują wartości NaN, null oraz wartości ujemne, oraz informacja o makysymalnej i minimalnej wartości w podzbiorze MODIS.

In [ ]:
df_modis = df_modis_vod[[c for c in df_modis_vod.columns if c.startswith('MODIS_')]]

negative_mask = (df_modis < 0)
any_negatives = negative_mask.any().any()
print("Czy istnieją wartości ujemne w VOD?", any_negatives)
print(f"Wartość maksymalna dla kolumn VOD: {df_modis.max().max()}")
print(f"Wartość minimalna dla kolumn VOD: {df_modis.min().min()}")

In [ ]:
modis_cols = [c for c in df_modis_vod.columns if c.startswith('MODIS_')]
neg_mask = (df_modis_vod[modis_cols] < 0)

df_modis[modis_cols] = df_modis_vod[modis_cols].mask(neg_mask, np.nan)

df_modis.isnull().sum().sum()

W tym przypadku, zgodnie z researchem, zdecydowałem się na usunięcie (zastąpienie ich wartościami 0) wartości ujemnych, ponieważ:
1. W normalnych warunkach odbicie nie powinno być ujemne, ponieważ fizycznie oznaczałoby to, że powierzchnia emituje więcej energii, niż otrzymuje, co jest niemożliwe w typowych warunkach
2. Ujemne wartości w praktyce są często traktowane jako artefakty przetwarzania danych lub błędy pomiarowe
3. Ujemne wartości większe od -0.001 są nieistotne i mogą wynikać z błędów numerycznych

In [ ]:
for col in modis_cols:
    df_modis[col].fillna(0, inplace=True)

Mamy bardzo dużo kolumn, dlatego pokazywanie histogramów w formie jak dla VOD było by nieczytelne. Poniżej boxplot

In [ ]:
plt.figure(figsize=(15, 15))
sns.boxplot(data=df_modis, orient='h')
plt.show()

Za dużo to nam nie mówi na razie, po za tym, że mamy niesymetreczny rozkład wartości.

## Heatmapa korelacji MODIS

In [ ]:
modis_corr = df_modis.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(modis_corr, cmap='viridis')
plt.title("Macierz korelacji MODIS (pasma i miesiące)")
plt.show()

Nadal ciężko z tak wielu kolumn wyciągnąć jakieś sensowne wnioski, natomiast z pomocą przychodzi wykres liniowy dla każdego pasma i avg miesiąca.

In [ ]:
modis_bands = sorted(set([col.split("_month_")[0] for col in df_modis.columns if "MODIS" in col]))

months = [f"m{i}" for i in range(1, 13)]

plt.figure(figsize=(12, 6))

for band in modis_bands:
    avg_month = []

    for m in months:
        # Tworzenie poprawnej nazwy kolumny dla danego bandu i miesiąca
        month_cols = [c for c in df_modis.columns if band in c and f"_month_{m}" in c]

        if len(month_cols) == 0:
            avg_month.append(None)
            continue

        col_data = df_modis[month_cols[0]]
        mean_val = col_data.mean(skipna=True)
        avg_month.append(mean_val)

    # Rysowanie wykresu dla danego bandu
    plt.plot(range(1, 13), avg_month, marker='o', label=band)

# Konfiguracja wykresu
plt.xticks(range(1, 13))  # miesiące na osi X
plt.xlabel("Miesiąc")
plt.ylabel("Średnia wartość MODIS")
plt.title("Sezonowe wahania MODIS dla różnych pasm")
plt.legend()
plt.show()

Można zauważyć praktycznie nakładające się na siebie dwie linie - band_01 i band_04. Decyduje się więc na scalenie tych dwóch w jeden band_14. Najpierw jednak sprawdzenie dokładne korelacji między tymi bandami.

In [ ]:
band_01_cols = [col for col in df_modis.columns if "band_01" in col]
band_04_cols = [col for col in df_modis.columns if "band_04" in col]

# Sprawdzamy, czy mamy tyle samo miesięcy w obu bandach
band_01_cols.sort()
band_04_cols.sort()

# Tworzymy DataFrame tylko z danymi tych dwóch bandów
df_band_01_04 = df_modis[band_01_cols].copy()
df_band_01_04.columns = [f"band_01_{i+1}" for i in range(len(band_01_cols))]  # Nazwy kolumn dla band_01
df_band_01_04 = df_band_01_04.join(df_modis[band_04_cols].set_axis([f"band_04_{i+1}" for i in range(len(band_04_cols))], axis=1))

# Obliczamy macierz korelacji
corr_matrix = df_band_01_04.corr()

print(corr_matrix)

# Wizualizacja macierzy korelacji
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Macierz korelacji między band_01 i band_04 dla wszystkich miesięcy")
plt.show()

To potwierdza zasadność scalenia tych bandów.

In [ ]:
months = [f"m{i}" for i in range(1, 13)]

for month in months:
    band_01_col = next((col for col in df_modis.columns if "band_01" in col and f"_month_{month}" in col), None)
    band_04_col = next((col for col in df_modis.columns if "band_04" in col and f"_month_{month}" in col), None)

    if band_01_col and band_04_col:
        # Tworzymy nową kolumnę jako średnią band_03 i band_04
        df_modis[f"MODIS_2000.2020_monthly_mean_surface_reflectance_band_14_._month_{month}"] = df_modis_vod[[band_01_col, band_04_col]].mean(axis=1)

        df_modis.drop(columns=[band_01_col, band_04_col], inplace=True)

df_modis.info()

### Wykres liniowy po scaleniu band01 i band04

In [ ]:
modis_bands = sorted(set([col.split("_month_")[0] for col in df_modis.columns if "MODIS" in col]))
months = [f"m{i}" for i in range(1, 13)]

plt.figure(figsize=(12, 6))

for band in modis_bands:
    avg_month = []

    for m in months:
        month_cols = [c for c in df_modis.columns if band in c and f"_month_{m}" in c]

        if len(month_cols) == 0:
            avg_month.append(None)
            continue

        col_data = df_modis[month_cols[0]]
        mean_val = col_data.mean(skipna=True)
        avg_month.append(mean_val)

    plt.plot(range(1, 13), avg_month, marker='o', label=band)

plt.xticks(range(1, 13))
plt.xlabel("Miesiąc")
plt.ylabel("Średnia wartość MODIS")
plt.title("Sezonowe wahania MODIS dla różnych pasm")
plt.legend()
plt.show()

## 6. Standaryzacja VOD
MinMaxScaler (0-1)

In [ ]:
vod_cols = [col for col in df_modis_vod_seasons.columns if "VOD" in col]
scaler_vod = MinMaxScaler()
df_modis_vod_seasons[vod_cols] = scaler_vod.fit_transform(df_modis_vod_seasons[vod_cols])

In [ ]:
for col in vod_cols:
    upper_limit = df_modis_vod_seasons[col].quantile(0.99)  # 99 percentyl
    lower_limit = df_modis_vod_seasons[col].quantile(0.01)  # 1 percentyl
    df_modis_vod[col] = df_modis_vod_seasons[col].clip(lower=lower_limit, upper=upper_limit)

### Wizualizacja VOD

In [ ]:
df_vod_last = df_modis_vod_seasons[[c for c in df_modis_vod_seasons.columns if c.startswith('VOD_')]]
plt.figure(figsize=(15, 8))
sns.boxplot(data=df_vod_last, orient='h')
plt.show()

## 7. Standaryzacja MODIS
Log Transform (log(x + 1))

In [ ]:
modis_cols = [col for col in df_modis.columns if "MODIS" in col]
df_modis[modis_cols] = np.log1p(df_modis[modis_cols])

In [ ]:
for col in modis_cols:
    upper_limit = df_modis[col].quantile(0.99)  # 99 percentyl
    lower_limit = df_modis[col].quantile(0.01)  # 1 percentyl
    df_modis[col] = df_modis[col].clip(lower=lower_limit, upper=upper_limit)

### Wizualizacja MODIS

In [ ]:
plt.figure(figsize=(15, 15))
sns.boxplot(data=df_modis, orient='h')
plt.show()